In [ ]:
from google.colab import files
files.upload()

Saving archive_17.zip to archive_17.zip


{'archive_17.zip': b'PK\x03\x04-\x00\x00\x00\x08\x00n\x86=O0*\x10\xce\xff\xff\xff\xff\xff\xff\xff\xff6\x00\x14\x00sentiment labelled sentences/amazon_cells_labelled.csv\x01\x00\x10\x00\r\xf2\x00\x00\x00\x00\x00\x00\xc4[\x00\x00\x00\x00\x00\x00\x95\xbd[{\x1bI\x92%\xf8\xae_\x11\xd2\xd4\x94\xa4^\x90ER\xa9\xaa\x9a~\xa9\x8f\x92(\x89\x95\x94\xc8")\xa9\xb2\xf7\xdb\x87\x00\x10\x00=\x19\x88@G\x04\x88D\xfe\xfa\xb5s\xcc\xcc\xdd\x03d\xce\xec\xf6CW\x8a\x00\xe2\xe2\x17\xb3c\xc7\x8e\x99\xdf\xb4\xc5pWuU\x11\xfa\xa2i\x8bm\xb9+\x16mW\xac\xaabh\x8bu\xbdY\x16a(BS\xe8w\x1a|\xb9\xf8vSl\x9a\xba\xea\xfb\xe2\xbcX\xb6\xc5tW\x94\xc5\xacm\x1e\xaan\xa8\xba\xc3\xc9\xd1D\xfe\xef\xd9\xa7\xb6\x9d\x17\xb3\xb2\xaf&\xc5\xd9o\xb3\xaa\xae\xabf(\x1e\xcazS\x1dN\x8e\xf9\x85\xae*\x07\xde\x0c\xd7\xfc\xb5\xdcN\xdb\xc6>\x9b<\xbb\r\xd5\x1cO0\xbb+\xbbe\xd5\xf1kz\x8b\xbe\x1cB\xdb\xf4E]\xf6Ch\x96\xc5\xaa\x95\x07\x1b\xee\xca\xa6\xf8\xe9m\xb1\n\xcdf\xa8\xfa\xc3/\xa7\xff\xbc\xbc.\xae\xae/\xdf]\x9c}\xb9y\xfe\xdc\x9e\xe9Vn\xb4\n3\xbc\xeb\

In [ ]:
import os
import zipfile
import pandas as pd

#mengekstrak file
local_zip = '/content/archive_17.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
os.listdir('/tmp/')

['sentiment labelled sentences',
 'dap_multiplexer.INFO',
 'initgoogle_syslog_dir.0',
 'debugger_2l1h4xcbfv',
 'dap_multiplexer.899649929fa2.root.log.INFO.20210826-024018.56']

In [ ]:
df = pd.read_csv('/tmp/sentiment labelled sentences/yelp_labelled.txt', names=['sentence','label'], sep='\t')

In [ ]:
df.tail()

,sentence,label
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


In [ ]:
df['sentence'] = df['sentence'].str.lower()

In [ ]:

# remove stopwords

#from nltk.corpus import stopwords #comment jika Error dan gunakan 2 sintaks dibawah
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,sentence,label
0,wow... loved place.,1
1,crust good.,0
2,tasty texture nasty.,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices.,1


In [ ]:
vocab_size = 2000
oov_tok = "<OOV>"
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' #remove symbols

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok, filters = filt)
tokenizer.fit_on_texts(df['sentence'].values)

word2index = tokenizer.word_index
print(len(word2index))

1998


In [ ]:
import json

with open('word2index.json', 'w') as fp:
    json.dump(word2index, fp)

In [ ]:

max_length =  max(len(values.split()) for i, values in enumerate(df['sentence']))
max_length

18

In [ ]:
trunc_type='post'

all_seq = tokenizer.texts_to_sequences(df['sentence'].values)
all_padded = pad_sequences(all_seq, maxlen = max_length, padding = trunc_type)
all_padded.shape

(1000, 18)

In [ ]:
# split train and test sets
from sklearn.model_selection import train_test_split

X = all_padded
#y = pd.get_dummies(df['label'].values)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#kalimat = df['sentence'].values
#y = df['label'].values

#kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, 
#                  

(800, 18) (800,)
(200, 18) (200,)


In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= vocab_size, output_dim=16, input_length= max_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 16)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 54,321
Trainable params: 54,321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/30
25/25 [==============================] - 3s 32ms/step - loss: 0.6946 - accuracy: 0.5063 - val_loss: 0.6941 - val_accuracy: 0.4800
Epoch 2/30
25/25 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5225 - val_loss: 0.6926 - val_accuracy: 0.5250
Epoch 3/30
25/25 [==============================] - 0s 11ms/step - loss: 0.6517 - accuracy: 0.6575 - val_loss: 0.5958 - val_accuracy: 0.7150
Epoch 4/30
25/25 [==============================] - 0s 12ms/step - loss: 0.3070 - accuracy: 0.9062 - val_loss: 0.6410 - val_accuracy: 0.7150
Epoch 5/30
25/25 [==============================] - 0s 11ms/step - loss: 0.1544 - accuracy: 0.9588 - val_loss: 0.6949 - val_accuracy: 0.7550
Epoch 6/30
25/25 [==============================] - 0s 12ms/step - loss: 0.0781 - accuracy: 0.9800 - val_loss: 0.7640 - val_accuracy: 0.7450
Epoch 7/30
25/25 [==============================] - 0s 12ms/step - loss: 0.0449 - accuracy: 0.9875 - val_loss: 0.8956 - val_accuracy: 0.7400
Epoch 8/30
25

In [ ]:

#def toSequence(sentence):
#  pad = []
#  for stc in sentence.split():
#    if stc.lower() in word2index.keys(): 
#      pad.append(word2index[stc.lower()])
#    else: 
#      continue
#  return pad

#pad = toSequence('affordable price and nice dessert')
#pad = [269, 353, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0 ,0,0,0,0]
#len(pad)
#model.predict([pad])

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [ ]:
saved_model_path = '/content/mymodel/'
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: /content/mymodel/assets


INFO:tensorflow:Assets written to: /content/mymodel/assets


In [ ]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  /content/mymodel/ \
  /content/modeltfjs

2021-08-26 02:46:58.422574: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-26 02:46:58.422629: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (899649929fa2): /proc/driver/nvidia/version does not exist
2021-08-26 02:47:00.153220: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-08-26 02:47:00.153460: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-08-26 02:47:00.162807: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 251 nodes (239), 348 edges (336), time = 4.688ms.
  function_optimizer: function_optimizer did nothing. time = 0.099ms.

2021-08-26 02:47:00.362783: I tensorflow/core/grappler/optimizers/meta_optim